In [2]:
import pandas as pd
import numpy as np
import csv

In [3]:
file = '../raw_data/HI-Small_Trans.csv'
df = pd.read_csv(file, decimal=',')

In [4]:
df.drop("Amount Paid", axis=1, inplace=True)

df = df.rename(columns={"Timestamp": "timestamp",
                   "From Bank": "from_bank",
                   "Account": "from_account",
                   "Account.1": "to_account",
                   "From Bank": "from_bank",
                   "To Bank": "to_bank",
                   "Amount Received": "amount",
                   "Payment Format": "payment_format",
                   "Is Laundering": "is_laundering"})


In [5]:
# dict for currency conversion

currency_dict = {"US Dollar": "USD",
                 "Bitcoin": "XBT",
                 "Euro": "EUR",
                 "Australiean Dollar": "AUD",
                 "Yuan": "CNY",
                 "Rupee": "INR",
                 "Yen": "JPY",
                 "Mexican Peso": "MXN",
                 "UK Pound": "GBP",
                 "Ruble": "RUB",
                 "Canadian Dollar": "CAD",
                 "Swiss Franc": "CHF",
                 "Brazil Real": "BRL",
                 "Saudi Riyal": "SAR",
                 "Shekel": "ILS"}

In [6]:
# put currency pair together and delete obsolete columns

df["Receiving Currency"] = df["Receiving Currency"].map(currency_dict)
df["Payment Currency"] = df["Payment Currency"].map(currency_dict)

df["currency_pair"] = df["Receiving Currency"] + "_" + df["Payment Currency"]
df = df.drop(['Receiving Currency', 'Payment Currency'], axis=1)

In [7]:
# list all unique accounts

from_accounts = df["from_account"].to_numpy()
to_accounts  = df["to_account"].to_numpy()

all_accounts = np.append(from_accounts, to_accounts)
all_accounts = np.unique(all_accounts)

In [14]:
# show only fraud

fraud_mask = df["is_laundering"] == 1

fraud_df = df[fraud_mask]

fraud_from = fraud_df["from_account"].to_numpy()
fraud_to  = fraud_df["to_account"].to_numpy()

all_fraud = np.append(fraud_from, fraud_to)
all_fraud = np.unique(all_fraud)

In [16]:
fraud_df

,timestamp,from_bank,from_account,to_bank,to_account,amount,payment_format,is_laundering,currency_pair
4742,2022/09/01 00:21,70,100428660,1124,800825340,389769.39,Cheque,1,USD_USD
40621,2022/09/01 00:03,70,100428660,11474,805B716C0,29024.33,Credit Card,1,USD_USD
69152,2022/09/01 00:01,70,100428660,15980,80B39E7B0,792.92,Credit Card,1,USD_USD
85763,2022/09/01 00:03,70,100428660,113798,80DC756E0,13171425.53,Cheque,1,USD_USD
90684,2022/09/01 00:23,70,100428660,32375,80E480620,14288.83,Cash,1,USD_USD
...,...,...,...,...,...,...,...,...,...
5071168,2022/09/10 23:29,111433,8065C2380,21550,8065C69E0,7773.42,ACH,1,USD_USD
5073375,2022/09/10 23:07,9,80727F930,219557,8072A28C0,448920.65,Cheque,1,RUB_RUB
5074248,2022/09/12 03:34,222,812D127D0,50202,812D129C0,20085.71,ACH,1,SAR_SAR
5075056,2022/09/10 23:43,17327,8095F6C20,15695,8095F6C70,3314.36,ACH,1,USD_USD
